In [ ]:
import torch

from transformers import AutoProcessor, AutoModelForVision2Seq, AutoTokenizer, AutoModel

from PIL import Image



# ==========================

# 1️⃣ Cấu hình thiết bị

# ==========================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")



# ==========================

# 2️⃣ Tải mô hình OCR (Viet-Wiki-Handwriting)

# ==========================

print("🔍 Đang tải mô hình OCR tiếng Việt viết tay...")

ocr_processor = AutoProcessor.from_pretrained("5CD-AI/Viet-Wiki-Handwriting")

ocr_model = AutoModelForVision2Seq.from_pretrained(

    "5CD-AI/Viet-Wiki-Handwriting",

    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32

).to(device).eval()



# ==========================

# 3️⃣ Tải mô hình ngôn ngữ (Vintern)

# ==========================

print("🧠 Đang tải mô hình Vintern...")

model = AutoModel.from_pretrained(

    "5CD-AI/Vintern-1B-v3_5",

    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,

    trust_remote_code=True

).eval().to(device)



tokenizer = AutoTokenizer.from_pretrained(

    "5CD-AI/Vintern-1B-v3_5",

    trust_remote_code=True,

    use_fast=False

)



# ==========================

# 4️⃣ Hàm xử lý ảnh

# ==========================

def process_image(image, question):

    # --- OCR ---

    print("🔍 Đang nhận dạng chữ viết tay...")

    inputs = ocr_processor(images=image, return_tensors="pt").to(device)

    with torch.inference_mode():

        generated_ids = ocr_model.generate(**inputs, max_new_tokens=512)

    ocr_text = ocr_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    print(f"📜 Văn bản OCR: {ocr_text}")



    # --- Vintern ---

    generation_config = {

        "max_new_tokens": 512,

        "do_sample": True,

        "temperature": 0.7,

        "top_p": 0.9,

        "repetition_penalty": 1.2

    }



    print("🧠 Đang phân tích với Vintern...")

    with torch.inference_mode():

        response, _ = model.chat(tokenizer, None, question, generation_config, history=None, return_history=True)

    return response

test_path = "test_dat.jpg"

test_image = Image.open(test_path)

out = process_image(test_image, "Trích xuất toàn bộ thông tin trong ảnh và trả về dạng Markdown.")

print(out)